In [3]:
import json
import os
import pickle
from shapely.geometry import shape, Point
import pandas as pd


"""
Use the store_object function to make local caching of the data.
"""
def store_object(filename, instruction):
    # check if file was cached before and read it
    if os.path.isfile(filename):
        return pickle.load(open(filename, 'rb'))
    # run instruction and save it as pickle then return object
    else:
        obj = instruction()
        pickle.dump(obj, open(filename, 'wb'))
        return obj
    
data = store_object("data.pkl", lambda: pd.read_csv("data.csv"))

with open("la_neighborhoods.geojson") as f:
    js = json.load(f)

In [29]:
# for every row in data check if it is in a neighborhood
from tqdm import tqdm

tqdm.pandas()

def get_neighborhood(row):
    # if row has column "neighborhood" return it
    if row["neighborhood"] is not None:
        return row["neighborhood"]

    point = Point(row["LON"], row["LAT"])
    if polygon.contains(point):
        return feature["properties"]["name"]
    return None


data = data[["DR_NO", "LON", "LAT"]]
data["neighborhood"] = None

for feature in js["features"]:
    polygon = shape(feature["geometry"])
    data["neighborhood"] = data.progress_apply(get_neighborhood, axis=1)

# drop other columns than neighborhood and DR_NO
data = data[["DR_NO", "neighborhood"]]

# output to new csv
data.to_csv("data_neighborhood.csv", index=False)


  0%|          | 0/901357 [00:00<?, ?it/s]

100%|██████████| 901357/901357 [00:05<00:00, 175560.59it/s]
